In [1]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_kernels

In [2]:
def construct_kernel(D, psi=8):
    locs = np.linspace(0,D/psi,D).reshape(-1,1)
    K = pairwise_kernels(locs, metric='rbf')
    return K

def construct_covariance(K):
    """Sigma_{ZZ} = E[Sigma]"""
    cov = K.copy()
    magic_const = 35 * np.sqrt(np.pi) / 64
    for i in range(cov.shape[0]):
        for j in range(cov.shape[1]):
            if i % 2 != j % 2:
                cov[i,j] *= magic_const
    return cov

def construct_cubed_covariance(K):
    """E[Sigma^3]"""
    cov3 = K.copy()**3
    magic_const_odd = 8 / 3
    magic_const_diff = 5 * np.sqrt(np.pi) / 8
    for i in range(cov.shape[0]):
        for j in range(cov.shape[1]):
            if i % 2 != j % 2:
                cov3[i,j] *= magic_const_diff
            elif (i+1) % 2 == 1:
                cov3[i,j] *= magic_const_odd
    return cov3

def construct_Sigma_ff(K):
    cov = construct_covariance(K)
    cov3 = construct_cubed_covariance(K)
    return 3 * (3*cov + 2*cov3)

def construct_Sigma_Zf(K):
    Sigma_Zf = 3*K
    magic_const_odd = 4 / 3
    magic_const_even_odd = 5 * np.sqrt(np.pi) / 8
    magic_const_odd_even = 35 * np.sqrt(np.pi) / 64
    for i in range(cov.shape[0]):
        for j in range(cov.shape[1]):
            if (i+1) % 2 == 1:
                if (j+1) % 2 == 1:
                    Sigma_Zf[i,j] *= magic_const_odd
                else:
                    Sigma_Zf[i,j] *= magic_const_odd_even
            elif (j+1) % 2 == 1:
                Sigma_Zf[i,j] *= magic_const_even_odd
    return Sigma_Zf

def nonzero_sparse_inds(k, D):
    inds = np.array([(i*(D+.5))//(k+1) for i in range(1,k+1)], dtype=int) - 1
    return inds

def get_beta(sparsity, D):
    if sparsity == 'dense':
        return 2**(2-np.arange(D)/2)
    if sparsity[-6:] != 'sparse':
        sys.exit('invalid sparsity type {}'.format(sparsity))
    k = int(sparsity[:-6])
    beta = np.zeros(D)
    inds = nonzero_sparse_inds(k, D)
    beta[inds] = 1
    return beta, inds+1

In [3]:
D = 10
K = construct_kernel(D)
cov = construct_covariance(K)
Sigma_ff = construct_Sigma_ff(K)
Sigma_Zf = construct_Sigma_Zf(K)
beta, comps = get_beta('1sparse', D)
bSffbp1 = 1 + beta.dot(Sigma_ff).dot(beta)
ELL = lambda s: -.5* (np.log(bSffbp1 - s) + 1 + np.log(2*np.pi))
SZfb = Sigma_Zf.dot(beta)

# larger score = larger expected log-likelihood
scores = SZfb**2 / np.diag(cov)
scores_arr = np.array([(i+1,s) for i, s in enumerate(scores)], 
                      dtype=[('ind', int), ('score', float)])
scores_arr.sort(order='score')
scores_arr = scores_arr[::-1]
best_ELL = ELL(scores_arr[0][-1])
print('1-sparse-nonlinear with D = {} and k_max = 1'.format(D))
print('best expected log-likelihood:', best_ELL)
print('data-generating beta components: ', *comps)
print()
print('model  best ELL - ELL')
for i, s in scores_arr:
    print('{:2d}     {:.4g}'.format(i, best_ELL - ELL(s)))

1-sparse-nonlinear with D = 10 and k_max = 1
best expected log-likelihood: -2.5702310797016956
data-generating beta components:  5

model  best ELL - ELL
 5     0
 7     0.103
 3     0.103
 6     0.215
 4     0.215
 9     0.2761
 1     0.2761
 8     0.2993
 2     0.2993
10     0.3894


In [4]:
top = 20
ks = [1, 2, 2]
Ds = [10, 20, 20]
psis = [8, 8, 4]

for (k, D, psi) in zip(ks, Ds, psis):
    K = construct_kernel(D, psi=psi)
    cov = construct_covariance(K)
    Sigma_ff = construct_Sigma_ff(K)
    Sigma_Zf = construct_Sigma_Zf(K)
    beta, comps = get_beta('{}sparse'.format(k), D)
    bSffbp1 = 1 + beta.dot(Sigma_ff).dot(beta)
    ELL = lambda s: -.5* (np.log(bSffbp1 - s) + 1 + np.log(2*np.pi))
    
    scores = {}
    for i in range(D):
        for j in range(i+1, D):
            SZfb = Sigma_Zf[(i,j),:].dot(beta)
#             if i == 4:
#                 print(cov[:,(i,j)])
#                 print(betaTcov)
#                 print(cov[np.ix_((i,j),(i,j))])
            scores[(i,j)] = np.linalg.solve(cov[np.ix_((i,j),(i,j))], SZfb).dot(SZfb)

    scores_arr = np.array([(i1+1,i2+1,s) for (i1,i2), s in scores.items()], 
                          dtype=[('ind1', int), ('ind2', int), ('score', float)])
    scores_arr.sort(order='score')
    scores_arr = scores_arr[::-1]
    best_ELL = ELL(scores_arr[0][-1])
    print('{}-sparse-nonlinear with D = {}, k_max = 2, psi = {}'.format(k, D, psi))
    print('best expected log-likelihood:', best_ELL)
    print('data-generating beta components: ', *comps)
    print()
    print(' model   best ELL - ELL')
    for ind, (i, j, s) in enumerate(scores_arr):
        print('{:2d}{:4d}   {:.4g}'.format(i, j,  best_ELL - ELL(s))) #, bSffbp1, s)
        if ind + 1 == top:
            break
    print()
    print()

1-sparse-nonlinear with D = 10, k_max = 2, psi = 8
best expected log-likelihood: -2.3760251587980212
data-generating beta components:  5

 model   best ELL - ELL
 7   8   0
 2   3   1.332e-15
 5   6   0.01048
 4   5   0.01048
 5   8   0.1609
 2   5   0.1609
 7  10   0.1693
 5  10   0.185
 5   9   0.1942
 5   7   0.1942
 3   5   0.1942
 1   5   0.1942
 3   7   0.2036
 7   9   0.2237
 1   3   0.2237
 3   9   0.2285
 1   7   0.2285
 6   7   0.2675
 3   4   0.2675
 9  10   0.2703


2-sparse-nonlinear with D = 20, k_max = 2, psi = 8
best expected log-likelihood: -2.781849754264274
data-generating beta components:  6 13

 model   best ELL - ELL
 7  13   0
 5  13   0.0004632
11  12   0.006617
 7  15   0.0143
 6  13   0.0205
 3  11   0.02102
 9  15   0.02119
 4  11   0.02176
 2  11   0.02433
 5  11   0.02447
 4  13   0.02548
13  14   0.02591
 9  17   0.03205
 9  13   0.03556
 1  11   0.03578
 6  11   0.0428
 7  11   0.04311
 3  13   0.04459
 9  19   0.06038
 9  11   0.06379


2-sparse-nonlinea